# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [3]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [4]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

movie_dict = {}
for p in movie_item:
    movie_dict.update({p["data-name"]: p["value"]})

In [5]:
movie_dict

{'復仇母親': '10392',
 '風暴過後': '10387',
 '南方車站的聚會': '10380',
 '祭旗': '10373',
 '唯你是愛': '10370',
 '屍控療法': '10367',
 '黑熊來了': '10357',
 '唐人街探案2': '10353',
 '黑計畫': '10351',
 'HUMAN LOST人間失格': '10348',
 '假面騎士劇場版ZI-O Over Quartzer': '10331',
 '巴黎小情聖': '10329',
 '每個月來訪一次的月經醬': '10324',
 '8級警戒': '10323',
 '為你存在的每一天': '10297',
 '黑色聖誕節': '10291',
 '熱氣球飛行家': '10257',
 '大說謊家': '10122',
 '特約經紀公司': '10364',
 '一屍到底番外篇：好萊塢大作戰！': '10352',
 '花生醬獵鷹的願望': '10340',
 '女孩要革命': '10339',
 '我的靈魂是愛做的': '10336',
 '絕世情歌': '10328',
 '人間失格：太宰治與他的3個女人': '10287',
 '抓狂博物館': '10279',
 '呆萌特務': '10264',
 '去年聖誕節': '10256',
 '野蠻遊戲：全面晉級': '10134',
 '夕霧花園': '10327',
 '懸案密碼前傳：瓶中信': '10254',
 '布魯克林孤兒': '10224',
 '暴走曼哈頓': '10044',
 '鋒迴路轉': '10258',
 '賽道狂人': '10097',
 '麂皮：永不滿足': '10358',
 '82年生的金智英': '10298',
 '冰雪奇緣2': '9597',
 'Hello World': '10302',
 '大餓': '10296',
 '我的情敵是冰箱': '10286',
 '萬萬沒想到': '10281',
 '為美好的世界獻上祝福！紅傳說': '10226',
 '山田孝之的痛苦與榮耀': '10301',
 '決戰中途島': '10130',
 '安眠醫生': '10098',
 '地久天長': '9621',
 '魔鬼終結者：黑暗宿命': '10067',
 

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [6]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 9597

In [7]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [8]:
# 指定放映地區
area_id = 8

In [9]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    #pass
    #date
    print("%s %s" % (date.p.string, date.h3.string))

十二月 15
十二月 16
十二月 17
十二月 18
十二月 19


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [10]:
# 選定要觀看的日期
date = "2019-12-14"

In [11]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [40]:
print('=' * 30)
for i in range(len(html)):
    print('電影院: ' + html[i].a.get_text())
    print('放映類型: ' + html[i].find(class_ = 'tapR').get_text())
    print('放映時刻: ')
    for j in range(len(html[i].find_all('input'))):
        print(html[i].find_all('input')[0].get('value'))
    print('=' * 30)

電影院: 鴻金寶麻吉影城
放映類型: 數位
放映時刻: 
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
2019-12-14 10:00:00
電影院: 三重天台戲院
放映類型: 數位
放映時刻: 
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
電影院: 國賓影城(中和環球購物中心)
放映類型: 數位
放映時刻: 
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
電影院: 板橋大遠百威秀影城
放映類型: 數位
放映時刻: 
2019-12-14 19:55:00
2019-12-14 19:55:00
2019-12-14 19:55:00
電影院: 板橋秀泰影城
放映類型: 數位
放映時刻: 
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-14 10:30:00
2019-12-